In [ ]:
!cat /proc/cpuinfo | grep model\ name

model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz


In [ ]:
import ast

def flat_object(obj, prefix=''):
    new_obj = {}
    for k, v in obj.items():
        if isinstance(v, dict):
            sub_obj = flat_object(v, prefix=f'{prefix}{k}_')
            new_obj = {**new_obj, **sub_obj}
        elif isinstance(v, str):
            try:
                elem = ast.literal_eval(v)
                if isinstance(elem, dict):
                    sub_obj = flat_object(elem, prefix=f'{prefix}{k}_')
                    new_obj = {**new_obj, **sub_obj}
                else:
                    new_obj[prefix + k] = elem
            except:
                new_obj[prefix + k] = v
        else:
            new_obj[prefix + k] = v
    return new_obj

In [ ]:
import json
import pandas as pd

def get_table(path):
    list_of_dicts = []
    with open(path, 'r') as f:
        for line in f:
            list_of_dicts.append(flat_object(json.loads(line)))
    df = pd.DataFrame.from_dict(list_of_dicts).dropna(axis=1, how='all')
    return df


In [ ]:
import hashlib

def md5hash(s: str):
    return hashlib.md5(str(s).encode('utf-8')).hexdigest()

def getmd5hash(prefix):
    return lambda x: md5hash(prefix + x)

## **business**

In [ ]:
business_df = get_table('/content/drive/MyDrive/Colab Notebooks/Yelp/yelp_academic_dataset_business.json')

business_df = business_df[business_df['categories'].notna()]
business_df['business_id'] = business_df['business_id'].apply(getmd5hash('business'))
business_df.insert(loc=0, column=':LABEL', value=['Business']*len(business_df))

In [ ]:
business_df.rename(columns={"business_id": "business_id:ID"}, errors='raise', inplace=True)

## **user**

In [ ]:
user_df = get_table('/content/drive/MyDrive/Colab Notebooks/Yelp/yelp_academic_dataset_user.json')
user_df.rename(columns={"user_id": "user_id:ID"}, errors='raise', inplace=True)
user_df.insert(loc=22, column=':LABEL', value=['User']*len(user_df))

## **review**

In [ ]:
review_df = get_table('/content/drive/MyDrive/Colab Notebooks/Yelp/yelp_academic_dataset_review.json')

In [ ]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
review_df['business_id'] = review_df['business_id'].apply(getmd5hash('business'))
review_df = review_df[review_df['business_id'].isin(business_df['business_id:ID'])]
review_df = review_df[review_df['user_id'].isin(user_df['user_id:ID'])]

review_df.insert(loc=0, column=':LABEL', value=['Review']*len(review_df))

# review_df['text'] = review_df['text'].str.replace('[",\\n,\\r]', '')
review_df['review_id'] = review_df['review_id'].apply(getmd5hash('review'))

In [ ]:
review_df.head()

,:LABEL,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Review,14b7eabcf7250cac8884af3677c0b8af,mh_-eMZ6K5RLWhZyISBhwA,dad4fb57fcb08795f8ecf02cf2ee3136,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,Review,0baa75e21e14c71f057e9ad453082867,OyoGAe7OKpv6SyGZT5g77Q,8b8c650e3345cb1b190161312ecc0967,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,Review,af722966fda3d69d36a0ac9de0292e51,8g_iMtfSiwikVnbP2etR0A,602a7a1a2e5e454d6195162b5989509e,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,Review,50d32c50663bd2b424f57561a09cbfda,_7bHUi9Uuf5__HHc_Q8guQ,caf1bdfefe5207b1260d47a2ab8a4b08,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Review,a3383e594a74583aabcda6de8e3cb8a8,bcjbaE6dDog4jkNY91ncLQ,6e5872cbd2865949eb95ef1de1b4c4c7,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
user_review_rels_df = review_df[['user_id', 'review_id', 'text', 'date']].copy()
# user_review_rels_df.insert(loc=2, column=':TYPE', value=['USER_REVIEW']*len(review_df))

In [ ]:
user_review_rels_df

,user_id,review_id,text,date
0,mh_-eMZ6K5RLWhZyISBhwA,14b7eabcf7250cac8884af3677c0b8af,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,OyoGAe7OKpv6SyGZT5g77Q,0baa75e21e14c71f057e9ad453082867,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,8g_iMtfSiwikVnbP2etR0A,af722966fda3d69d36a0ac9de0292e51,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,_7bHUi9Uuf5__HHc_Q8guQ,50d32c50663bd2b424f57561a09cbfda,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,bcjbaE6dDog4jkNY91ncLQ,a3383e594a74583aabcda6de8e3cb8a8,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...
6990275,qskILQ3k0I_qcCMI-k6_QQ,94e309064b7859538c514528e80db157,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,Zo0th2m8Ez4gLSbHftiQvg,2d0aa703beebc1996c58711a6e4b04e3,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,mm6E4FbCMwJmb7kPDZ5v2Q,eaac0ea0e1fb9aab851008575a49c98c,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,YwAMC-jvZ1fvEUum6QkEkw,0dab15fea0a38d0f706afeed71945194,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
user_review_rels_df['text'] = user_review_rels_df['text'].astype(str)

In [ ]:
user_review_rels_df['sentiment_score'] =user_review_rels_df['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

In [ ]:
user_review_rels_df.to_csv('/content/drive/MyDrive/user_review_rels.csv', index=False)

In [ ]:
user_review_rels_df

,user_id,review_id,text,date,sentiment_score
0,mh_-eMZ6K5RLWhZyISBhwA,14b7eabcf7250cac8884af3677c0b8af,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.8597
1,OyoGAe7OKpv6SyGZT5g77Q,0baa75e21e14c71f057e9ad453082867,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,0.9858
2,8g_iMtfSiwikVnbP2etR0A,af722966fda3d69d36a0ac9de0292e51,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,0.9201
3,_7bHUi9Uuf5__HHc_Q8guQ,50d32c50663bd2b424f57561a09cbfda,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.9588
4,bcjbaE6dDog4jkNY91ncLQ,a3383e594a74583aabcda6de8e3cb8a8,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.9804
...,...,...,...,...,...
6990275,qskILQ3k0I_qcCMI-k6_QQ,94e309064b7859538c514528e80db157,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,0.1027
6990276,Zo0th2m8Ez4gLSbHftiQvg,2d0aa703beebc1996c58711a6e4b04e3,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,0.8549
6990277,mm6E4FbCMwJmb7kPDZ5v2Q,eaac0ea0e1fb9aab851008575a49c98c,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,0.6792
6990278,YwAMC-jvZ1fvEUum6QkEkw,0dab15fea0a38d0f706afeed71945194,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,0.9982


## **生成边文件**

In [1]:
import pandas as pd
import torch
user_ids = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Yelp/user_ids.csv')
review_ids = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Yelp/review_ids.csv')

user_ids['user_pos_id'] = user_ids.index
review_ids['review_pos_id'] = review_ids.index

review_user_rels = pd.read_csv('/content/drive/MyDrive/user_review_rels.csv')

In [2]:
review_user_rels

,user_id,review_id,text,date,sentiment_score
0,mh_-eMZ6K5RLWhZyISBhwA,14b7eabcf7250cac8884af3677c0b8af,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0.8597
1,OyoGAe7OKpv6SyGZT5g77Q,0baa75e21e14c71f057e9ad453082867,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,0.9858
2,8g_iMtfSiwikVnbP2etR0A,af722966fda3d69d36a0ac9de0292e51,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,0.9201
3,_7bHUi9Uuf5__HHc_Q8guQ,50d32c50663bd2b424f57561a09cbfda,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,0.9588
4,bcjbaE6dDog4jkNY91ncLQ,a3383e594a74583aabcda6de8e3cb8a8,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,0.9804
...,...,...,...,...,...
6989555,qskILQ3k0I_qcCMI-k6_QQ,94e309064b7859538c514528e80db157,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,0.1027
6989556,Zo0th2m8Ez4gLSbHftiQvg,2d0aa703beebc1996c58711a6e4b04e3,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,0.8549
6989557,mm6E4FbCMwJmb7kPDZ5v2Q,eaac0ea0e1fb9aab851008575a49c98c,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,0.6792
6989558,YwAMC-jvZ1fvEUum6QkEkw,0dab15fea0a38d0f706afeed71945194,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,0.9982


In [3]:
user_review_rels_time = review_user_rels[['user_id', 'review_id', 'date', 'sentiment_score']].copy()

In [4]:
user_review_rels_time

,user_id,review_id,date,sentiment_score
0,mh_-eMZ6K5RLWhZyISBhwA,14b7eabcf7250cac8884af3677c0b8af,2018-07-07 22:09:11,0.8597
1,OyoGAe7OKpv6SyGZT5g77Q,0baa75e21e14c71f057e9ad453082867,2012-01-03 15:28:18,0.9858
2,8g_iMtfSiwikVnbP2etR0A,af722966fda3d69d36a0ac9de0292e51,2014-02-05 20:30:30,0.9201
3,_7bHUi9Uuf5__HHc_Q8guQ,50d32c50663bd2b424f57561a09cbfda,2015-01-04 00:01:03,0.9588
4,bcjbaE6dDog4jkNY91ncLQ,a3383e594a74583aabcda6de8e3cb8a8,2017-01-14 20:54:15,0.9804
...,...,...,...,...
6989555,qskILQ3k0I_qcCMI-k6_QQ,94e309064b7859538c514528e80db157,2014-12-17 21:45:20,0.1027
6989556,Zo0th2m8Ez4gLSbHftiQvg,2d0aa703beebc1996c58711a6e4b04e3,2021-03-31 16:55:10,0.8549
6989557,mm6E4FbCMwJmb7kPDZ5v2Q,eaac0ea0e1fb9aab851008575a49c98c,2019-12-30 03:56:30,0.6792
6989558,YwAMC-jvZ1fvEUum6QkEkw,0dab15fea0a38d0f706afeed71945194,2022-01-19 18:59:27,0.9982


In [5]:
user_review_rels = pd.merge(pd.merge(user_review_rels_time, user_ids, left_on='user_id', right_on='user_id:ID'), review_ids, left_on='review_id', right_on='review_id:ID')

In [6]:
user_review_rels

,user_id,review_id,date,sentiment_score,user_id:ID,user_pos_id,review_id:ID,review_pos_id
0,mh_-eMZ6K5RLWhZyISBhwA,14b7eabcf7250cac8884af3677c0b8af,2018-07-07 22:09:11,0.8597,mh_-eMZ6K5RLWhZyISBhwA,124234,14b7eabcf7250cac8884af3677c0b8af,0
1,mh_-eMZ6K5RLWhZyISBhwA,a7314f2acfc11c4531ddbd77effa97d6,2017-03-05 14:26:14,0.8379,mh_-eMZ6K5RLWhZyISBhwA,124234,a7314f2acfc11c4531ddbd77effa97d6,45632
2,mh_-eMZ6K5RLWhZyISBhwA,332c6c81bd8b3fcb9566442add52f2b0,2018-04-25 17:04:09,0.9697,mh_-eMZ6K5RLWhZyISBhwA,124234,332c6c81bd8b3fcb9566442add52f2b0,264558
3,mh_-eMZ6K5RLWhZyISBhwA,6b36c9710154c217adfeb672c9d29277,2017-02-12 20:12:42,0.9224,mh_-eMZ6K5RLWhZyISBhwA,124234,6b36c9710154c217adfeb672c9d29277,822507
4,mh_-eMZ6K5RLWhZyISBhwA,02d9398a829783005f777b5570d5eabd,2017-03-02 22:03:57,0.9462,mh_-eMZ6K5RLWhZyISBhwA,124234,02d9398a829783005f777b5570d5eabd,925818
...,...,...,...,...,...,...,...,...
6989553,Is5IWDXWtqVMCVtedjJIQQ,181d9374ca3a48b98c957c2a1fdbbb99,2018-11-09 18:09:35,0.9837,Is5IWDXWtqVMCVtedjJIQQ,1849913,181d9374ca3a48b98c957c2a1fdbbb99,6989527
6989554,99_EbpwCcCRAa66YgorrKA,a0383d50dd7cf0720088367c7008a14c,2016-04-27 04:14:52,-0.7506,99_EbpwCcCRAa66YgorrKA,1908824,a0383d50dd7cf0720088367c7008a14c,6989536
6989555,-AkziDwQ8hv2COTDBBUpig,7c3f4650cc068c25378d70ff768cacff,2022-01-18 03:48:44,0.6003,-AkziDwQ8hv2COTDBBUpig,1897371,7c3f4650cc068c25378d70ff768cacff,6989543
6989556,s67G457QlHSvk5RjOMN91w,13ecf95eba8387b2bbbad2a8f9a26302,2022-01-18 15:24:44,0.8832,s67G457QlHSvk5RjOMN91w,1969447,13ecf95eba8387b2bbbad2a8f9a26302,6989548


## **创建user-review的边，计算边的权重**


In [19]:
import numpy as np

In [7]:
# 转换date格式
user_review_rels['date'] = pd.to_datetime(user_review_rels['date'])

In [8]:
user_review_rels_w = user_review_rels[['user_pos_id', 'review_pos_id', 'sentiment_score', 'date' ]].copy()

In [10]:
user_review_rels_w.head()

,user_pos_id,review_pos_id,sentiment_score,date
0,124234,0,0.8597,2018-07-07 22:09:11
1,124234,45632,0.8379,2017-03-05 14:26:14
2,124234,264558,0.9697,2018-04-25 17:04:09
3,124234,822507,0.9224,2017-02-12 20:12:42
4,124234,925818,0.9462,2017-03-02 22:03:57


In [12]:
# 获取用户的交互数据
user_concat = user_review_rels_w['user_pos_id'].unique()
user_concat

array([ 124234,   51327,   98159, ..., 1897371, 1969447, 1938727])

In [14]:
selected_users_interactions = user_review_rels_w[user_review_rels_w['user_pos_id'].isin(user_concat)]

In [16]:
# 获取选出用户的最早交互时间
earliest_interaction_time = selected_users_interactions.groupby('user_pos_id')['date'].min()

In [17]:
earliest_interaction_time

user_pos_id
0         2009-04-26 22:28:18
1         2010-10-11 15:46:45
2         2009-03-09 08:00:05
3         2008-07-22 01:35:24
4         2015-10-17 01:59:38
                  ...        
1987892   2018-02-27 02:09:23
1987893   2019-07-05 20:34:24
1987894   2017-02-05 07:05:21
1987895   2020-05-10 18:38:38
1987896   2021-08-04 04:29:52
Name: date, Length: 1987653, dtype: datetime64[ns]

**计算权重**

In [18]:
# 假设时间衰减因子gamma为0.001
gamma = 0.001
# 设置阈值为1小时
threshold = pd.Timedelta(hours=168)

In [20]:
# 计算时间差并应用时间衰减函数和阈值
selected_users_interactions['time_difference'] = selected_users_interactions.apply(lambda row: (row['date'] - earliest_interaction_time[row['user_pos_id']]).total_seconds(), axis=1)
selected_users_interactions['time_decay'] = np.exp(-gamma * selected_users_interactions['time_difference'])
selected_users_interactions['time_decay'] = np.where(selected_users_interactions['time_difference'] > threshold.total_seconds(), 0.01, selected_users_interactions['time_decay'])

# 计算每个用户与项目的权重
selected_users_interactions['edge_weight'] = selected_users_interactions['sentiment_score'] * selected_users_interactions['time_decay']

print("每个用户与项目的权重:")
print(selected_users_interactions)

每个用户与项目的权重:
         user_pos_id  review_pos_id  sentiment_score                date  \
0             124234              0           0.8597 2018-07-07 22:09:11   
1             124234          45632           0.8379 2017-03-05 14:26:14   
2             124234         264558           0.9697 2018-04-25 17:04:09   
3             124234         822507           0.9224 2017-02-12 20:12:42   
4             124234         925818           0.9462 2017-03-02 22:03:57   
...              ...            ...              ...                 ...   
6989553      1849913        6989527           0.9837 2018-11-09 18:09:35   
6989554      1908824        6989536          -0.7506 2016-04-27 04:14:52   
6989555      1897371        6989543           0.6003 2022-01-18 03:48:44   
6989556      1969447        6989548           0.8832 2022-01-18 15:24:44   
6989557      1938727        6989553           0.7835 2022-01-17 20:59:01   

         time_difference  time_decay  edge_weight  
0             47093565.

In [22]:
selected_users_interactions.head(50)

,user_pos_id,review_pos_id,sentiment_score,date,time_difference,time_decay,edge_weight
0,124234,0,0.8597,2018-07-07 22:09:11,47093565.0,1.000000e-02,8.597000e-03
1,124234,45632,0.8379,2017-03-05 14:26:14,4816188.0,1.000000e-02,8.379000e-03
2,124234,264558,0.9697,2018-04-25 17:04:09,40768063.0,1.000000e-02,9.697000e-03
3,124234,822507,0.9224,2017-02-12 20:12:42,3022576.0,1.000000e-02,9.224000e-03
4,124234,925818,0.9462,2017-03-02 22:03:57,4584451.0,1.000000e-02,9.462000e-03
5,124234,1077217,0.9734,2018-10-29 21:54:59,56942313.0,1.000000e-02,9.734000e-03
6,124234,1168357,0.9864,2018-02-11 03:07:30,34410664.0,1.000000e-02,9.864000e-03
7,124234,1573839,0.9156,2017-01-08 20:45:48,562.0,5.700678e-01,5.219541e-01
8,124234,2129050,0.1695,2018-01-21 18:08:11,32650305.0,1.000000e-02,1.695000e-03
9,124234,2150684,0.9889,2017-01-29 15:48:51,1797145.0,1.000000e-02,9.889000e-03
